# 1. Import the necessary libraries for you to be able to query 4square API

In [1]:
import requests
import json
import pandas as pd
from getpass import getpass
import os
import time

# Get FSQ token from .env file
from dotenv import load_dotenv
load_dotenv() # load_env
token = os.getenv("token")




In [2]:

def requests_for_foursquare (query, lat, lon, radius=500, limit=1):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    
    try:
        return requests.get(url, headers=headers).json()
    except:
        print("no :(")
## ------------------------------------------------------------------

def gets_coordinates_from_string (where):
    url_geocode = f"https://geocode.xyz/{where}?json=1"
    res = requests.get(url_geocode)
    
    try:
        return res.json()
    
    except:
        time.sleep(10)
        gets_coordinates_from_string (where)

res = gets_coordinates_from_string ("96 Pamplona, Barcelona, Spain")
res

{'standard': {'stnumber': '96',
  'addresst': 'Pamplona',
  'statename': 'Catalonia',
  'postal': '08018',
  'region': 'Catalonia',
  'city': 'Barcelona',
  'prov': 'ES',
  'countryname': 'Spain',
  'confidence': '1.00'},
 'longt': '2.19035',
 'alt': {},
 'elevation': {},
 'latt': '41.39780'}

In [3]:
lat, lon = res["latt"], res["longt"]
print(lat,lon)

41.39780 2.19035


# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [4]:
res = requests_for_foursquare ("coffee", lat, lon, radius=5000, limit=10)
res2 = res["results"]
df = pd.DataFrame(res2)
coffee_df = df[['name','location']]
coffee_df

,name,location
0,Nømad Roaster's Home,"{'address': 'Calle de Pujades, 95', 'admin_reg..."
1,Three Marks Coffee,"{'address': 'Ausiàs Marc, 151', 'admin_region'..."
2,Brew Coffee,"{'address': 'Calle de Roger de Flor, 102', 'ad..."
3,El Petit Príncep,"{'address': 'Gran Vía de Les Corts Catalanes, ..."
4,Federal,"{'address': 'Carrer del Taulat, 109', 'admin_r..."
5,Nømad Coffee Lab & Shop,"{'address': 'Pasatge Sert, 12', 'admin_region'..."
6,Funky Bakers,"{'address': 'Carrer de la Diputació, 347', 'ad..."
7,BRKLYN Torrefaction,"{'address': 'Calle de Bac de Roda, 79', 'admin..."
8,El Born,"{'address': 'Passeig del Born, 26', 'admin_reg..."
9,Bornet,"{'address': 'Barra de Ferro, 3', 'admin_region..."


# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [5]:
res = requests_for_foursquare ("coffee", lat, lon, limit=5)
res2 = res["results"]
df = pd.DataFrame(res2)
coffee_df = df[['name','location']]
coffee_df

,name,location
0,SKYE Coffee Co,"{'address': 'Calle Pamplona, 88', 'admin_regio..."
1,Syra Coffee Poblenou,"{'address': 'Calle de Pujades, 100', 'admin_re..."
2,Espai Joliu,"{'address': 'Carrer Badajoz, 95', 'admin_regio..."
3,Nømad Roaster's Home,"{'address': 'Calle de Pujades, 95', 'admin_reg..."
4,Eva-2,"{'address': 'Bolivia, 21', 'admin_region': 'Ca..."


# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [9]:
def foursquare_places(venue="Starbucks or restaurants" ,coordinates = '41.39780,2.19035'):

    url = "https://api.foursquare.com/v3/places/search?"

    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    params = {
        'll': coordinates,
        'query': "starbucks,restaurants",
        'limit': 20
    }
    response = requests.get(url, params=params,headers=headers)

    data = response.json()
    venues = data['results']

    # Extract data into a list of dictionaries
    results = []
    for venue in venues:
        name = venue['name']
        lat = venue['geocodes']['main']['latitude']
        lng = venue['geocodes']['main']['longitude']
        results.append({'name': name, 'lat': lat, 'lng': lng})

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(results)

    return df

foursquare_places()



,name,lat,lng
0,Starbucks,41.383476,2.171057
1,Starbucks,41.392939,2.165386
2,Starbucks,41.388022,2.167940
3,Starbucks,41.376510,2.150584
4,Starbucks,41.385779,2.167932
5,Starbucks,41.395808,2.159836
6,Starbucks,41.402337,2.174068
7,Starbucks,41.374808,2.182781
8,Starbucks,41.381925,2.172230
9,Starbucks,41.390753,2.165031


# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [10]:
df_out = pd.DataFrame(foursquare_places())
df_out.to_json("../lab-coffee-shops/data/coffe_shops.json")

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [ ]:
# your code here